In [1]:
#load everything
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from tqdm import tqdm

from operator import attrgetter

dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#assume given bill number, type, and congress
bill_number = '4387'
bill_type = 'hr'
congress = '113'

#load bill
query = "SELECT * FROM features_legis WHERE bill_type LIKE '"+bill_type+"' AND bill_number LIKE '"+bill_number+"' AND congress = '"+congress+"' ;" #should only be one match.
bill_legis = pd.read_sql_query(query,engine) #note to self, bills repeat, I only want the most recent one.
#drop useless columns.
bill_legis = bill_legis.drop(['bill_type','bill_number','congress','num_amends','index','status','result','final_result'],1)
#read in legislator info
legis = pd.read_sql_table('sponsor_list_det',engine)

In [3]:
cols = bill_legis.columns.tolist()
mat = bill_legis.as_matrix().flatten().tolist()
print(len(mat))
print(len(cols))

869
869


In [4]:
#list of legislators on bill
legislators = list()
for leg in range(0,len(cols)):
    if mat[leg] == 1:
        legislators.append(cols[leg])
print(legislators)

['Rep,"Bachus, Spencer",AL', 'Rep,"Cotton, Tom",AR', 'Rep,"Fincher, Stephen Lee",TN', 'Rep,"Garrett, Scott",NJ', 'Rep,"Hultgren, Randy",IL', 'Rep,"Hurt, Robert",VA', 'Rep,"Mulvaney, Mick",SC', 'Rep,"Peters, Gary C.",MI', 'Rep,"Pittenger, Robert",NC', 'Rep,"Ross, Dennis A.",FL', 'Rep,"Rothfus, Keith J.",PA', 'Rep,"Stivers, Steve",OH', 'Rep,"Wagner, Ann",MO', 'Rep,"Westmoreland, Lynn A.",GA']


In [6]:
#loop through legislators and give each two dataframes, linking contributors and amounts
legis_contribs = list()
legis_names = list()
for leg in tqdm(legislators): #should be fast
    #first get row in legis
    legis_ind = legis[legis.sponsor == leg]
    ind_legis_contrib = contributions.pull_contributions(legis_ind,legis,engine)
    legis_names.append(legis_ind.qsponsor.iloc[0])
    u_reg = np.unique(ind_legis_contrib.registrantid)
    reg_dict = {'rid':list(), 'name':list(), 'amount':list()}
    for rid in u_reg:
        matches = ind_legis_contrib[ind_legis_contrib.registrantid == rid]
        reg_dict['rid'].append(rid)
        reg_dict['name'].append(matches.registrantname.iloc[0])
        reg_dict['amount'].append(np.sum(matches.amount))
    legis_contribs.append(pd.DataFrame.from_dict(reg_dict))    #dataframe with contributor names and id's

100%|██████████| 14/14 [00:12<00:00,  1.18it/s]


In [26]:
ind_legis_contrib = contributions.pull_contributions(leg_ind,all_legis,engine)

In [27]:
ind_legis_contrib.iloc[0]

index                                   163
registrantname      The Grossman Group, LLC
received                2009-04-08 10:43:07
year                                   2008
amount                                  500
honoree               Rep. Neil Abercrombie
contributiondate        2008-06-30 00:00:00
registrantid                          87814
contributiontype                       FECA
Name: 0, dtype: object

In [28]:
#get all unique contributors using rid
u_reg = np.unique(ind_legis_contrib.registrantid) #all unique contributors

In [34]:
#loop over rid and sum up totals.
#summed_ind_legis_contrib = pd.DataFrame(columns=str(u_reg))
reg_name_dict = {'rid':list(), 'name':list()}
reg_money_dict = {'rid':list(), 'amount':list()}

#contrib_names = pd.DataFrame(columns=['rid','name'])
#contrib_amount = pd.DataFrame(columns=['rid','amount'])
for rid in u_reg:
    matches = ind_legis_contrib[ind_legis_contrib.registrantid == rid]
    reg_name_dict['rid'].append(rid)
    reg_name_dict['name'].append(matches.registrantname.iloc[0])
    reg_money_dict['rid'].append(rid)
    reg_money_dict['amount'].append(np.sum(matches.amount))
contrib_names = pd.DataFrame.from_dict(reg_name_dict)    #dataframe with contributor names and id's
contrib_amount = pd.DataFrame.from_dict(reg_money_dict)  #dataframe with contributor id's and donation totals

In [8]:
#loop through legislators and get funding df for each one, stored in a list.
legis_funds = list()
legis_names = list()
for leg in legislators:
    #first get row in legis
    legis_ind = legis[legis.sponsor == leg]
    if legis_ind['index'].size > 1: #something has gone wrong.
        print('Something has gone wrong.')
        break
    legis_funds.append(contributions.pull_contributions(legis_ind,legis,engine))
    legis_names.append(legis_ind.qsponsor.iloc[0])

Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen


In [9]:
print(len(legis_funds))
print(legis_names)
print(legis_names[0])

14
['Rep, Bachus,  Spencer', 'Rep, Cotton,  Tom', 'Rep, Fincher,  Stephen Lee', 'Rep, Garrett,  Scott', 'Rep, Hultgren,  Randy', 'Rep, Hurt,  Robert', 'Rep, Mulvaney,  Mick', 'Rep, Peters,  Gary C.', 'Rep, Pittenger,  Robert', 'Rep, Ross,  Dennis A.', 'Rep, Rothfus,  Keith J.', 'Rep, Stivers,  Steve', 'Rep, Wagner,  Ann', 'Rep, Westmoreland,  Lynn A.']
Rep, Bachus,  Spencer


In [ ]:
allfunds = list()
allcontribs = list()
#do what I need with one dataframe first
counter = 0
for df in legis_funds :
    #sum up contributions from the same contributers
    u_reg = np.unique(df.registrantid) #all unique contributors
    allcontribs.append(u_reg)
    u_reg_name = list() #name of contributor
    u_reg_con = list() #sum of contributions
    for rid_ind in range(0,len(u_reg)):
        regid = u_reg[rid_ind] #id for contributor
        regname = df[df.registrantid == regid]['registrantname'].iloc[0]
        tot_money = np.sum(df[df.registrantid == regid]['amount'])
        u_reg_name.append(regname)
        u_reg_con.append(tot_money)
    u_reg = u_reg.tolist()
    temp_name = legis_names[counter]
    temp_name = [temp_name]*len(u_reg)
    allfunds.append(list(zip(u_reg,u_reg_name,u_reg_con,temp_name)))
    print(allfunds[0][0])
    counter = counter+1 

# Put all of the above into a table for each legislator. Thus I have all of the unique contributions summed for each legislator.

In [1]:
#load everything
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from tqdm import tqdm

from operator import attrgetter

dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

Populating the interactive namespace from numpy and matplotlib


## Step 1: Get list of unique legislators.

In [2]:
legis = pd.read_sql_table('sponsor_list_det',engine)
#all_legis

In [3]:
#loop through legislators and give each two dataframes, linking contributors and amounts
#legis_contribs = list()
#legis_names = list()

#make a table for each legislator with contribution totals
for leg in tqdm(range(0,len(legis.sponsor))): #should be fast
    #first get row in legis
    legis_ind = legis[legis.index == leg]
    tableid = 'legis'+str(leg) #table id for each legislator
    #legis_ind = legis[legis.sponsor == leg]
    ind_legis_contrib = contributions.pull_contributions(legis_ind,legis,engine)
    legis_names.append(legis_ind.qsponsor.iloc[0])
    u_reg = np.unique(ind_legis_contrib.registrantid)
    reg_dict = {'rid':list(), 'name':list(), 'amount':list()}
    for rid in u_reg:
        matches = ind_legis_contrib[ind_legis_contrib.registrantid == rid]
        reg_dict['rid'].append(rid)
        reg_dict['name'].append(matches.registrantname.iloc[0])
        reg_dict['amount'].append(np.sum(matches.amount))
    towrite = pd.DataFrame.from_dict(reg_dict)
    sqlCommands.write_to_database(dbname,tableid,towrite,engine)
#legis_contribs.append(pd.DataFrame.from_dict(reg_dict))    #dataframe with contributor names and id's

100%|██████████| 389/389 [08:02<00:00,  1.15s/it]


## Step 2: Try with a bill. Pull the tables of legislators.

In [1]:
#load everything
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from tqdm import tqdm

from operator import attrgetter

dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

Populating the interactive namespace from numpy and matplotlib


Get legislators on bill.

In [2]:
#assume given bill number, type, and congress
bill_number = '4387'
bill_type = 'hr'
congress = '113'

#load bill
query = "SELECT * FROM features_legis WHERE bill_type LIKE '"+bill_type+"' AND bill_number LIKE '"+bill_number+"' AND congress = '"+congress+"' ;" #should only be one match.
bill_legis = pd.read_sql_query(query,engine) #note to self, bills repeat, I only want the most recent one.
#drop useless columns.
bill_legis = bill_legis.drop(['bill_type','bill_number','congress','num_amends','index','status','result','final_result'],1)
#read in legislator info
legis = pd.read_sql_table('sponsor_list_det',engine)

cols = bill_legis.columns.tolist()
mat = bill_legis.as_matrix().flatten().tolist()

#list of legislators on bill
legislators = list()
for leg in range(0,len(cols)):
    if mat[leg] == 1:
        legislators.append(cols[leg])

In [3]:
print(legislators)

['Rep,"Bachus, Spencer",AL', 'Rep,"Cotton, Tom",AR', 'Rep,"Fincher, Stephen Lee",TN', 'Rep,"Garrett, Scott",NJ', 'Rep,"Hultgren, Randy",IL', 'Rep,"Hurt, Robert",VA', 'Rep,"Mulvaney, Mick",SC', 'Rep,"Peters, Gary C.",MI', 'Rep,"Pittenger, Robert",NC', 'Rep,"Ross, Dennis A.",FL', 'Rep,"Rothfus, Keith J.",PA', 'Rep,"Stivers, Steve",OH', 'Rep,"Wagner, Ann",MO', 'Rep,"Westmoreland, Lynn A.",GA']


In [4]:
legis_funds = list() #will be a list to hold the dataframes of contributions for each legislator
legis_names = list()
for leg in legislators:
    #get index of legislator
    legis_ind = legis[legis.sponsor == leg].index[0]
    legis_names.append(legis[legis.sponsor == leg]['qsponsor'].iloc[0])    
    #generate tableid
    tableid = 'legis'+str(legis_ind)
    #read in the contribution table for this legislator and append to list.
    legis_funds.append(pd.read_sql_table(tableid,engine))


In [5]:
legis_names

['Rep, Bachus,  Spencer',
 'Rep, Cotton,  Tom',
 'Rep, Fincher,  Stephen Lee',
 'Rep, Garrett,  Scott',
 'Rep, Hultgren,  Randy',
 'Rep, Hurt,  Robert',
 'Rep, Mulvaney,  Mick',
 'Rep, Peters,  Gary C.',
 'Rep, Pittenger,  Robert',
 'Rep, Ross,  Dennis A.',
 'Rep, Rothfus,  Keith J.',
 'Rep, Stivers,  Steve',
 'Rep, Wagner,  Ann',
 'Rep, Westmoreland,  Lynn A.']

In [6]:
#now need to get shared contributors. Do this by concatinating rid's into a big list and keeping only ones that are duplicates.
#loop through and make a new dataframe with just the amount, rid, and name.
all_dfs = pd.DataFrame()
for df in legis_funds:
    all_dfs = pd.concat([all_dfs,df[['amount','name','rid']]],axis=0)

In [7]:
all_dfs

,amount,name,rid
0,5000.0,"Adams and Reese, LLP",157
1,5000.0,Advanta Corporation,237
2,2500.0,EADS NORTH AMERICA,330
3,48000.0,AFLAC INCORPORATED,385
4,8000.0,AKIN GUMP STRAUSS HAUER & FELD LLP,682
5,32500.0,Alabama Farmers Federation,693
6,10000.0,Allstate Insurance Company,1114
7,1000.0,Time Warner Inc.,1295
8,2000.0,AMERICAN AIRLINES,1422
9,44000.0,American Bankers Association,1785


In [8]:
#get duplicate (shared) contributors based on rid
duplicates = all_dfs[all_dfs.duplicated(subset='rid',keep=False)]
#get unique rid's of shared contributors.
u_shared_rids = np.unique(duplicates['rid'])
#get the names of the unique rids
u_shared_names = list()
for rid in u_shared_rids:
    u_shared_names.append(duplicates[duplicates.rid==rid]['name'].iloc[0])

In [9]:
#now I need to calculate influence each rid had on a legislator. So first find intersection of each legislator with unique ids
shared_funds = list()
for df in legis_funds:
    shared_contribs = df[df['rid'].isin(u_shared_rids)]
    #get total amount of money in shared_contribs
    total_contribs = sum(shared_contribs.amount.tolist())
    #create a new column in shared contribs with influence (ratio of money to total money).
    shared_contribs['influence'] = (shared_contribs.amount/total_contribs)*100.
    shared_funds.append(shared_contribs)

/home/lordluen/anaconda2/envs/py3_env/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


This is the slowest step. I could not think of a good way to avoid for loops here.

In [21]:
#calculate top 10 contributors based on influence.
total_inf = np.zeros(len(u_shared_rids))
total_money = np.zeros(len(u_shared_rids))

all_df = pd.concat(shared_funds)
for i in tqdm(range(0,len(u_shared_rids))):
    rid = u_shared_rids[i]
    match = all_df[all_df.rid == rid]
    total_inf[i] = total_inf[i]+sum(match['influence'])
    total_money[i] = total_money[i]+sum(match['amount'])
avg_inf = total_inf/len(shared_funds)
avg_money = total_money/len(shared_funds)


100%|██████████| 861/861 [00:00<00:00, 1863.95it/s]


In [18]:
#calculate top 10 contributors based on influence.
total_inf = np.zeros(len(u_shared_rids))
total_money = np.zeros(len(u_shared_rids))

for i in tqdm(range(0,len(u_shared_rids))):
    rid = u_shared_rids[i]
    for j in shared_funds:
        match = j[j.rid==rid]
        if len(match) > 0:
            total_inf[i] = total_inf[i] + match['influence'].iloc[0]
            total_money[i] = total_money[i] + match['amount'].iloc[0]
avg_inf = total_inf/len(shared_funds)
avg_money = total_money/len(shared_funds)

100%|██████████| 861/861 [00:04<00:00, 174.00it/s]


In [19]:
print(total_inf[0])
print(total_money[0])

4.45027558167
126500.0


In [105]:
#zip together u_shared_rids, u_shared_names, total_inf, total_money
shared_con_zip = list(zip(u_shared_rids.tolist(),u_shared_names,avg_inf.tolist(),avg_money.tolist()))
#sort based on influence
shared_con_zip = sorted(shared_con_zip, key=lambda tup: tup[2], reverse=True)

In [106]:
print(shared_con_zip[0:10])

[(4766, 'ASSOCIATED BUILDERS AND CONTRACTORS, INC.', 1.8108117943352957, 33442.857142857145), (316890, 'DELOITTE LLP', 1.5744774886214574, 48607.142857142855), (53824, 'CAPITAL ONE FINANCIAL CORPORATION', 1.4546108715652266, 32478.571428571428), (1785, 'American Bankers Association', 1.4353381684202504, 31642.85714285714), (400390816, 'Koch Companies Public Sector, LLC', 1.2639333036764409, 26892.85714285714), (19228, 'INDEPENDENT INSURANCE AGENTS & BROKERS OF AMERICA', 1.1617397697312928, 27250.0), (27274, 'National Beer Wholesalers Association', 1.0952773134601446, 25067.85714285714), (10764, 'PRICEWATERHOUSECOOPERS', 1.0506148121143197, 27107.14285714286), (27263, 'NATIONAL AUTOMOBILE DEALERS ASSOCIATION', 1.0140530211306809, 22785.714285714286), (2354, 'American Financial Services Association', 1.0121407270247014, 19946.428571428572)]


# Final step is to generate a file from this data given an rid.

In [115]:
contrib_tup = (shared_con_zip,shared_funds,legis_names)
rank = 0

In [116]:
print(legis_names)

['Rep, Bachus,  Spencer', 'Rep, Cotton,  Tom', 'Rep, Fincher,  Stephen Lee', 'Rep, Garrett,  Scott', 'Rep, Hultgren,  Randy', 'Rep, Hurt,  Robert', 'Rep, Mulvaney,  Mick', 'Rep, Peters,  Gary C.', 'Rep, Pittenger,  Robert', 'Rep, Ross,  Dennis A.', 'Rep, Rothfus,  Keith J.', 'Rep, Stivers,  Steve', 'Rep, Wagner,  Ann', 'Rep, Westmoreland,  Lynn A.']


In [129]:
#first get rid for contributor in question.
rid = contrib_tup[0][rank][0]
#loop through legislators and just make list of funds and influence.
funds = list()
inf = list()
for j in contrib_tup[1]:
    match = j[j.rid == rid]
    if len(match) > 0:
        funds.append(match['amount'].iloc[0])
        inf.append(match['influence'].iloc[0])
    else:
        funds.append(0.)
        inf.append(0.)

In [132]:
print(funds)
print(inf)
print(contrib_tup[2])


[17500.0, 40200.0, 43500.0, 35000.0, 70000.0, 39500.0, 33500.0, 0.0, 20000.0, 42500.0, 37500.0, 52000.0, 15000.0, 22000.0]
[0.40277165542485333, 2.8624026185885754, 1.9364559330739464, 0.92926581050062196, 3.9663531991627483, 1.5561861747230445, 2.5887038099607653, 0.0, 2.4743032473399138, 1.2983973725423026, 4.5168155804554289, 1.08984922625494, 0.67772142052271223, 1.0521390721442818]
['Rep, Bachus,  Spencer', 'Rep, Cotton,  Tom', 'Rep, Fincher,  Stephen Lee', 'Rep, Garrett,  Scott', 'Rep, Hultgren,  Randy', 'Rep, Hurt,  Robert', 'Rep, Mulvaney,  Mick', 'Rep, Peters,  Gary C.', 'Rep, Pittenger,  Robert', 'Rep, Ross,  Dennis A.', 'Rep, Rothfus,  Keith J.', 'Rep, Stivers,  Steve', 'Rep, Wagner,  Ann', 'Rep, Westmoreland,  Lynn A.']


In [128]:
match['influence'].iloc[0]

0.40277165542485333